In [1]:
import pandas as pd
import torch.nn as nn
import pickle
import torch
from torchvision import models
from torchvision.models import detection, resnet50, ResNet50_Weights
import os
import numpy as np
import cv2
from torchvision import transforms
import pymc3 as pm
import theano.tensor as tt
from sklearn.preprocessing import LabelEncoder
import scipy

In [2]:
CONFIGS = {
    # determine the current device and based on that set the pin memory
    # flag
    "DEVICE": "cuda" if torch.cuda.is_available() else "cpu",
    # specify ImageNet mean and standard deviation
    "IMG_MEAN": [0.485, 0.456, 0.406],
    "IMG_STD": [0.229, 0.224, 0.225],
    "MC_DROPOUT_ENABLED": False,  # Switch to enable/disable MC Dropout for confidence score
    "NUM_DROPOUT_RUNS": 3,
    "CONFIDENCE_THRESHOLD": 0,
    "BIG_MODEL_IMG_SIZE": 320,
    "SMALL_MODEL_IMG_SIZE": 60,
    "MEAN_PRIOR": -15,
}

# Big model

## Model loading

In [3]:
class MultiHeadResNet_BigModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_BigModel, self).__init__()
        self.base_model = models.resnet50(weights=ResNet50_Weights.DEFAULT)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)
        self.fc_bbox = nn.Linear(num_ftrs, 4)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        box = self.fc_bbox(x)
        return prdtype, weight, halal, healthy, box

    
# load label encoder 
def load_label_encoder_big_model():
    le_prdtype = pickle.loads(open("../../NN_model/model_weights/traindatawithin1/le_prdtype.pickle", "rb").read())
    le_weight = pickle.loads(open("../../NN_model/model_weights/traindatawithin1/le_weight.pickle", "rb").read())
    le_halal = pickle.loads(open("../../NN_model/model_weights/traindatawithin1/le_halal.pickle", "rb").read())
    le_healthy = pickle.loads(open("../../NN_model/model_weights/traindatawithin1/le_healthy.pickle", "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy

le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_big_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_BigModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = '../../NN_model/model_weights/traindatawithin1/multi_head_model.pth'
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model

big_model = load_model()

/Users/liupeng/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.2.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


## Scoring on main imgs

In [4]:
main_imgs_results_big_model = pd.read_csv("../../NN_model/model_weights/traindatawithin1/main_imgs_results_big_model.csv")
main_imgs_results_big_model.head()

,Filename,CorrectTotalLabel,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,IMG_9696_JPG.rf.275e6f14bf9fdf35973c5b9fa6da01...,RolledOatsInstantOatmeal_900-999g_Halal_Healthy,-2.903158,-1.158375,-2.541948,-1.342361,-2.808456,-1.799497,-2.676470,-2.128480,...,-1.735707,-0.312206,-0.561187,-1.566575,-0.674635,4.956939,1.345725,-0.719949,1.197672,-0.627469
1,IMG_1997_jpeg.rf.e54c43e25598c549364cc51d313db...,BiscuitsCrackersCookies_1-99g_NonHalal_NonHealthy,-1.477122,-2.052021,-2.475709,-2.197139,7.244976,-2.088915,-2.557024,-2.251854,...,-0.218654,-0.289336,-1.117378,-0.645007,-0.799627,-0.580764,-0.461047,0.950722,-0.897013,0.606925
2,IMG_6938_jpeg.rf.ca6186a9f8255d8468f116972b651...,Flour_1000-1999g_Halal_NonHealthy,-1.487290,-2.360942,-1.221453,-1.374544,-0.850634,-1.876865,-1.990161,-1.781574,...,-0.638138,-0.732298,-0.515879,-0.564577,-0.439350,-0.664563,1.059038,-0.344334,-0.652783,0.949639
3,IMG_5615_jpeg.rf.cd89865c8097c5c4a2a022b40fd9b...,InstantNoodles_400-499g_Halal_NonHealthy,-1.706056,-1.597529,-2.038913,-2.214169,-0.924510,-2.062986,-1.467856,-1.985458,...,4.575842,-0.614573,-0.786767,-1.016497,-0.526474,-0.871664,0.925577,-0.590882,-1.103862,0.668659
4,IMG_5612_jpeg.rf.05bf9b066415edaab68bf7fdc28c6...,InstantNoodles_400-499g_Halal_NonHealthy,-1.294374,-1.565522,-1.925313,-1.123500,-0.819286,-1.715673,-1.586196,-1.858472,...,4.853828,-0.513871,-0.664848,-1.077270,-0.697597,-0.854779,0.959387,-0.677899,-1.004177,0.481502


In [5]:
# Create a copy of the current column names to a list
new_columns = main_imgs_results_big_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
main_imgs_results_big_model.columns = new_columns


In [6]:
main_imgs_results_big_model.shape

(3720, 98)

# Scoring on unused training imgs

In [7]:
master_df = pd.read_csv("../../master_list.csv")
master_df.head()

,filepath,xmin,ymin,xmax,ymax,label,ProductType,Weight,HalalStatus,HealthStatus,new_camera,tag
0,IMG_20230428_123528_jpg.rf.5687b7b914f6d9aa98c...,151,42,497,591,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
1,IMG_20230428_123522_jpg.rf.204ff37f497f2dce442...,88,81,442,567,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
2,IMG_20230428_123708_jpg.rf.141ecd0cefaea75c0b7...,35,34,492,622,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
3,IMG_20230428_123521_jpg.rf.1069b402272252862ec...,99,122,428,587,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN
4,IMG_20230428_123659_jpg.rf.5e1b6c4caabe48cf360...,103,17,474,592,Sugar_400-499g_NonHalal_NonHealthy,Sugar,400-499g,NonHalal,NonHealthy,0,NaN


In [8]:
a = set(main_imgs_results_big_model["filepath"].tolist())
b = set(master_df["filepath"].tolist())
pending_imgs = [i for i in b if i not in a]
len(pending_imgs)

734

In [9]:
transforms_test = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
])

new_imgs_results_big_model = []  # List to store the results

for row in pending_imgs:
    image_path = "../../rshiny/www/all_images/" + row
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['BIG_MODEL_IMG_SIZE'], CONFIGS['BIG_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4, _ = big_model(frame)

    # reference the correct label from master list
    tmp_correct_label = master_df.loc[master_df['filepath'] == row, 'label'].iloc[0]
        
    # Extract and store the results
    prediction_row = [row, tmp_correct_label]
    prediction_row.extend(out1.cpu().numpy().flatten())
    prediction_row.extend(out2.cpu().numpy().flatten())
    prediction_row.extend(out3.cpu().numpy().flatten())
    prediction_row.extend(out4.cpu().numpy().flatten())
    new_imgs_results_big_model.append(prediction_row)


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
big_model_pred_col_name_original = main_imgs_results_big_model.columns[2:].tolist()
column_names += big_model_pred_col_name_original

# Create the DataFrame
new_imgs_results_big_model = pd.DataFrame(new_imgs_results_big_model, columns=column_names)
new_imgs_results_big_model.head()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,...,Weight_400-499g,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy
0,20231222_0790.jpg,OtherSauceDressing_200-299g_NonHalal_NonHealthy,-2.383856,-3.003223,-1.365573,-1.305672,-1.972551,-2.533299,-2.282753,-1.424270,...,-1.116435,1.090512,-0.511903,-0.328206,-0.814076,-0.752357,0.783849,-0.244295,-0.743399,0.467866
1,IMG_0765_jpeg.rf.947b67a360f2fb534a0d21c119a2a...,InstantMeals_100-199g_Halal_NonHealthy,-2.654079,-2.213080,-1.229503,-1.339858,-3.208222,-2.591826,-2.093502,-2.188624,...,-0.303540,-0.560444,-0.678913,-0.923080,-0.808220,-0.203618,0.996058,-0.732323,-1.141674,0.674551
2,20240123_2_1095.jpg,TeaPowderLeaves_100-199g_Halal_NonHealthy,-2.095917,-2.198940,-1.255963,-0.402960,-0.317007,-1.914161,-0.483683,-1.107384,...,0.770428,0.071614,-0.411436,-1.002624,-0.703624,-0.419791,-0.677719,0.607387,-0.782379,0.830414
3,IMG_6541_jpeg.rf.e76ff9b17f614823f12c31fe2722b...,HoneyOtherSpreads_400-499g_Halal_NonHealthy,-2.871182,-2.265496,-2.338285,-1.178075,-1.625538,-2.345879,-3.055430,-2.160223,...,0.987425,-0.160367,-1.046913,-0.828945,-1.106665,-0.590752,0.156297,0.099564,-1.067681,0.796956
4,2023_10_25_12_21_7_211924.jpg,Babyfood_100-199g_NonHalal_NonHealthy,-0.605847,-0.378154,-2.886815,-3.223178,-0.298598,-2.405183,-2.047013,-1.847265,...,1.027581,-1.602003,-1.113279,-0.762784,-0.206645,-0.555985,0.958457,0.240950,-1.069329,1.026830


## Scoring on new imgs

In [10]:
# new_imgs_df = pd.read_csv("GPT_model/chatgpt_prediction.csv")
# new_imgs_df.head()

In [11]:
# transforms_test = transforms.Compose([
#     transforms.ToPILImage(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=CONFIGS['IMG_MEAN'], std=CONFIGS['IMG_STD'])
# ])

In [12]:
# master_df = pd.read_csv("../../master_list.csv")
# master_df.head()

In [13]:
# new_imgs_results_big_model = []  # List to store the results

# for idx, row in new_imgs_df.iterrows():
#     image_path = "../../all_images/" + row['img_filename']
#     frame = cv2.imread(image_path)

#     # Preprocessing steps
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     frame = cv2.resize(frame, (CONFIGS['BIG_MODEL_IMG_SIZE'], CONFIGS['BIG_MODEL_IMG_SIZE']))
#     frame = frame.transpose((2, 0, 1))
#     frame = torch.from_numpy(frame).float()
#     frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

#     # Perform prediction
#     with torch.no_grad():
#         out1, out2, out3, out4, _ = big_model(frame)

#     # reference the correct label from master list
#     tmp_correct_label = master_df.loc[master_df['filepath'] == row['img_filename'], 'label'].iloc[0]
        
#     # Extract and store the results
#     prediction_row = [row['img_filename'], tmp_correct_label]
#     prediction_row.extend(out1.cpu().numpy().flatten())
#     prediction_row.extend(out2.cpu().numpy().flatten())
#     prediction_row.extend(out3.cpu().numpy().flatten())
#     prediction_row.extend(out4.cpu().numpy().flatten())
#     new_imgs_results_big_model.append(prediction_row)


# # Define column names for the new DataFrame
# column_names = ['filepath', 'label']
# big_model_pred_col_name_original = main_imgs_results_big_model.columns[2:].tolist()
# column_names += big_model_pred_col_name_original

# # Create the DataFrame
# new_imgs_results_big_model = pd.DataFrame(new_imgs_results_big_model, columns=column_names)
# new_imgs_results_big_model.head()

In [14]:
# new_imgs_results_big_model.shape

## All scorings from big model

In [15]:
main_imgs_results_big_model['img_type'] = "existing"
new_imgs_results_big_model['img_type'] = "new"
all_imgs_results_big_model = pd.concat([main_imgs_results_big_model, new_imgs_results_big_model], axis=0)
all_imgs_results_big_model.reset_index(drop=True, inplace=True)
all_imgs_results_big_model.head()
# all_imgs_results_big_model = main_imgs_results_big_model

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
0,IMG_9696_JPG.rf.275e6f14bf9fdf35973c5b9fa6da01...,RolledOatsInstantOatmeal_900-999g_Halal_Healthy,-2.903158,-1.158375,-2.541948,-1.342361,-2.808456,-1.799497,-2.676470,-2.128480,...,-0.312206,-0.561187,-1.566575,-0.674635,4.956939,1.345725,-0.719949,1.197672,-0.627469,existing
1,IMG_1997_jpeg.rf.e54c43e25598c549364cc51d313db...,BiscuitsCrackersCookies_1-99g_NonHalal_NonHealthy,-1.477122,-2.052021,-2.475709,-2.197139,7.244976,-2.088915,-2.557024,-2.251854,...,-0.289336,-1.117378,-0.645007,-0.799627,-0.580764,-0.461047,0.950722,-0.897013,0.606925,existing
2,IMG_6938_jpeg.rf.ca6186a9f8255d8468f116972b651...,Flour_1000-1999g_Halal_NonHealthy,-1.487290,-2.360942,-1.221453,-1.374544,-0.850634,-1.876865,-1.990161,-1.781574,...,-0.732298,-0.515879,-0.564577,-0.439350,-0.664563,1.059038,-0.344334,-0.652783,0.949639,existing
3,IMG_5615_jpeg.rf.cd89865c8097c5c4a2a022b40fd9b...,InstantNoodles_400-499g_Halal_NonHealthy,-1.706056,-1.597529,-2.038913,-2.214169,-0.924510,-2.062986,-1.467856,-1.985458,...,-0.614573,-0.786767,-1.016497,-0.526474,-0.871664,0.925577,-0.590882,-1.103862,0.668659,existing
4,IMG_5612_jpeg.rf.05bf9b066415edaab68bf7fdc28c6...,InstantNoodles_400-499g_Halal_NonHealthy,-1.294374,-1.565522,-1.925313,-1.123500,-0.819286,-1.715673,-1.586196,-1.858472,...,-0.513871,-0.664848,-1.077270,-0.697597,-0.854779,0.959387,-0.677899,-1.004177,0.481502,existing


In [16]:
new_imgs_results_big_model.shape

(734, 99)

In [17]:
main_imgs_results_big_model.shape

(3720, 99)

In [18]:
all_imgs_results_big_model.tail()

,filepath,label,ProductType_AdultMilk,ProductType_BabyMilkPowder,ProductType_Babyfood,ProductType_BeehoonVermicelli,ProductType_BiscuitsCrackersCookies,ProductType_Book,ProductType_BreakfastCereals,ProductType_CannedBakedBeans,...,Weight_500-599g,Weight_600-699g,Weight_700-799g,Weight_800-899g,Weight_900-999g,HalalStatus_Halal,HalalStatus_NonHalal,HealthStatus_Healthy,HealthStatus_NonHealthy,img_type
4449,4311699431913_.pic.jpg,BiscuitsCrackersCookies_1-99g_NonHalal_NonHealthy,-2.216774,-1.041516,-0.189125,-0.703690,3.130214,-1.948151,-1.811058,-1.779595,...,-0.742330,-1.119186,-0.695860,-0.400297,-0.754505,0.218350,0.141504,-0.828839,0.693758,new
4450,20231222_0227.jpg,OtherDriedFood_100-199g_NonHalal_NonHealthy,-2.025125,-2.263761,-1.635945,-2.032518,-0.851909,-2.239129,-1.715036,-1.960197,...,-0.762524,-1.016977,-0.738850,-0.219820,-0.113523,0.506712,0.318041,-0.400186,0.184633,new
4451,Crackers_300-399g_0311_Halal_6_png.rf.38873347...,BiscuitsCrackersCookies_300-399g_Halal_NonHealthy,-1.238606,-1.123214,-2.143519,-1.597176,4.489266,-2.235688,-1.271442,-1.868534,...,0.270248,-0.933225,-0.520792,-0.039021,-1.365222,0.688315,-0.057608,-0.971977,0.956312,new
4452,20231222_0220.jpg,OtherDriedFood_100-199g_NonHalal_NonHealthy,-2.288204,-2.365383,-1.590779,-1.131465,-1.190371,-3.257054,-1.798546,-1.989489,...,-0.911950,-1.112657,-0.738925,-0.085185,-0.532475,0.186526,0.497743,-0.609307,0.492602,new
4453,Crackers_400-499g_0311_NonHalal_5_png.rf.0f30f...,BiscuitsCrackersCookies_400-499g_NonHalal_NonH...,-1.757106,-1.157677,-1.873900,-2.431889,7.551000,-1.875409,-2.185850,-1.940819,...,-0.478862,-0.176279,-1.114221,-0.029811,-0.717680,-0.016947,0.613798,-0.807794,0.835908,new


In [19]:
all_imgs_results_big_model.shape

(4454, 99)

In [20]:
all_imgs_results_big_model.to_csv("../../NN_model/model_weights/traindatawithin1/all_imgs_results_big_model.csv", index=True)


In [21]:
100*51+150*52

12900

In [22]:
300*52

15600

In [23]:
class MultiHeadResNet_SmallModel(nn.Module):
    def __init__(self, num_classes_prdtype, num_classes_weight, num_classes_halal, num_classes_healthy):
        super(MultiHeadResNet_SmallModel, self).__init__()
        self.base_model = models.resnet18(pretrained=True)
        num_ftrs = self.base_model.fc.in_features
        self.base_model.fc = nn.Identity()

        # Define custom fully connected layers for each prediction head
        self.fc_prdtype = nn.Linear(num_ftrs, num_classes_prdtype)
        self.fc_weight = nn.Linear(num_ftrs, num_classes_weight)
        self.fc_halal = nn.Linear(num_ftrs, num_classes_halal)
        self.fc_healthy = nn.Linear(num_ftrs, num_classes_healthy)

    def forward(self, x):
        x = self.base_model(x)
        prdtype = self.fc_prdtype(x)
        weight = self.fc_weight(x)
        halal = self.fc_halal(x)
        healthy = self.fc_healthy(x)
        return prdtype, weight, halal, healthy

    
# load label encoder 
def load_label_encoder_small_model():
    le_prdtype = pickle.loads(open("../small_model/output/le_prdtype.pickle", "rb").read())
    le_weight = pickle.loads(open("../small_model/output/le_weight.pickle", "rb").read())
    le_halal = pickle.loads(open("../small_model/output/le_halal.pickle", "rb").read())
    le_healthy = pickle.loads(open("../small_model/output/le_healthy.pickle", "rb").read())
    
    return le_prdtype, le_weight, le_halal, le_healthy

le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

# Load the trained MultiHeadResNet model
def load_model():
    # Verify the number of classes for each label
    num_classes_prdtype = len(le_prdtype.classes_)
    num_classes_weight = len(le_weight.classes_)
    num_classes_halal = len(le_halal.classes_)
    num_classes_healthy = len(le_healthy.classes_)
    # print(num_classes_prdtype)
    # print(num_classes_healthy)

    custom_resnet_model = MultiHeadResNet_SmallModel(
        num_classes_prdtype=num_classes_prdtype,
        num_classes_weight=num_classes_weight,
        num_classes_halal=num_classes_halal,
        num_classes_healthy=num_classes_healthy
    )

    model_path = '../small_model/output/multi_head_model.pth'
    # print("test1")
    if os.path.exists(model_path):
        custom_resnet_model.load_state_dict(torch.load(model_path, map_location=CONFIGS['DEVICE']))
    else:
        raise FileNotFoundError(f"Model file not found: {model_path}")
    # print("test2")
    custom_resnet_model.to(CONFIGS['DEVICE'])
    custom_resnet_model.eval()
    return custom_resnet_model
 
small_model = load_model()

FileNotFoundError: [Errno 2] No such file or directory: '../small_model/output/le_prdtype.pickle'

In [ ]:
new_imgs_df = pd.read_csv("../small_model/new_imgs_list.csv")
new_imgs_df.reset_index(drop=True, inplace=True)

# ADHOC: change the new imgs to existing type
new_imgs_df['label'] = 'AdultMilk_1-99g_Halal_NonHealthy'
new_imgs_df['ProductType'] = 'AdultMilk'
new_imgs_df['Weight'] = '1-99g'
new_imgs_df['HalalStatus'] = 'Halal'
new_imgs_df['HealthStatus'] = 'NonHealthy'

new_imgs_df.head()

In [ ]:
new_imgs_results_small_model = pd.read_csv("../small_model/new_imgs_results_small_model.csv")
new_imgs_results_small_model = new_imgs_results_small_model.loc[new_imgs_results_small_model.Filename.isin(new_imgs_df.filepath)]
new_imgs_results_small_model.reset_index(drop=True, inplace=True)
new_imgs_results_small_model.head()

In [ ]:
new_imgs_results_small_model.shape

In [ ]:
# Create a copy of the current column names to a list
new_columns = new_imgs_results_small_model.columns.tolist()

# Modify the first two elements
new_columns[0] = 'filepath'
new_columns[1] = 'label'

# Assign the modified list of column names back to the DataFrame
new_imgs_results_small_model.columns = new_columns
new_imgs_results_small_model.head()

In [ ]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_imgs_results_big_model.columns) - set(new_imgs_results_small_model.columns))

if len(new_prdtype)>0:
    for col in new_prdtype:
        new_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=new_imgs_results_small_model.shape[0])  # Initialize new columns

new_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

In [ ]:
new_imgs_results_small_model.shape

In [ ]:
main_imgs_master_list = pd.read_csv("../master_list.csv")
main_imgs_master_list.head()

In [ ]:
main_imgs_results_small_model = []  # List to store the results
le_prdtype, le_weight, le_halal, le_healthy = load_label_encoder_small_model()

for idx, row in main_imgs_master_list.iterrows():
    image_path = "../all_images/" + row['filepath']
    frame = cv2.imread(image_path)

    # Preprocessing steps
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (CONFIGS['SMALL_MODEL_IMG_SIZE'], CONFIGS['SMALL_MODEL_IMG_SIZE']))
    frame = frame.transpose((2, 0, 1))
    frame = torch.from_numpy(frame).float()
    frame = transforms_test(frame).unsqueeze(0).to(CONFIGS['DEVICE'])

    # Perform prediction
    with torch.no_grad():
        out1, out2, out3, out4 = small_model(frame)
    
    # Extract and store the results
    prediction_row = [row['filepath'], row['label']]
    prediction_row.extend(out1.cpu().numpy().flatten())
    prediction_row.extend(out2.cpu().numpy().flatten())
    prediction_row.extend(out3.cpu().numpy().flatten())
    prediction_row.extend(out4.cpu().numpy().flatten())
    main_imgs_results_small_model.append(prediction_row)


# Define column names for the new DataFrame
column_names = ['filepath', 'label']
column_names += ['ProductType_' + name for name in le_prdtype.classes_]
column_names += ['Weight_' + name for name in le_weight.classes_]
column_names += ['HalalStatus_' + name for name in le_halal.classes_]
column_names += ['HealthStatus_' + name for name in le_healthy.classes_]


# Create the DataFrame
main_imgs_results_small_model = pd.DataFrame(main_imgs_results_small_model, columns=column_names)
main_imgs_results_small_model.head()

In [ ]:
main_imgs_results_small_model.shape

In [ ]:
# Check if any name from 'extracted_names' is not in 'df' and add it as a new column
new_prdtype = list(set(all_imgs_results_big_model.columns) - set(main_imgs_results_small_model.columns))

if len(new_prdtype)>0:
    for col in new_prdtype:
        main_imgs_results_small_model[col] = np.random.normal(loc=CONFIGS["MEAN_PRIOR"], scale=np.sqrt(0.1), size=main_imgs_results_small_model.shape[0])  # Initialize new columns

main_imgs_results_small_model.head()  # Display the updated DataFrame for verificatio

In [ ]:
main_imgs_results_small_model.shape

In [ ]:
main_imgs_results_small_model['img_type'] = "existing"
new_imgs_results_small_model['img_type'] = "new"
all_imgs_results_small_model = pd.concat([main_imgs_results_small_model, new_imgs_results_small_model], axis=0)
all_imgs_results_small_model.reset_index(drop=True, inplace=True)
all_imgs_results_small_model.head()

In [ ]:
all_imgs_results_small_model.tail()

In [ ]:
all_imgs_results_small_model.shape

In [ ]:
all_imgs_results_small_model.to_csv("all_imgs_results_small_model.csv", index=True)

In [ ]:
0.08*0.6+0.14*0.4

In [ ]:
(0.6**2) * (0.12**2) + (0.4**2) * (0.18**2) + 2*0.6*0.4*0.12*0.18*0.6